In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import folium
from folium.plugins import HeatMap
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import numpy as np, matplotlib.pyplot as plt, seaborn as sns



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
crimes=pd.read_csv("../input/crimes-in-boston/crime.csv",encoding = "ISO-8859-1")
crimes.head()

* Now we can try to understand our columns a little bit. When we call crimes.head(), we can see an example of the data with first 5 rows. First things that can be reliazed are:
* INCIDENT_NUMBER is the record number of the incident.
* OFFENSE_CODE is the code for crime.
* OFFENSE_CODE_GROUP seems to be a more general category which includes different OFFENSE_CODEs, we will look at it later. It also seems to be a general explanation of the incident.
* OFFENSE_DESCRIPTION seems to be a more understandable, more self explanatory column.
* DISTRICT areas are most probably areas of Boston, neighbors in a way.
* REPORTING_AREA seems to be a more spesific location than district which where the incidents was reported.
* SHOOTING is to explain whether was there any shootings in incident or not.
* OCCURED_ON_DATE, YEAR, MONTH, DAY_OF_WEEK, HOUR STREET, Lat, Long columns are self explanatory.
* UCR_PART is a mystery at this point which we will search for more information.

Let's make column names lowercase so that we won't be thinking which one was uppercase and which one was lower. Plus it is easier to write in lowercase.

In [ ]:
crimes.columns = map(str.lower, crimes.columns)
crimes.head()

Let's find out how many duplicates df has to heck and eventually remove them.

In [ ]:
print(crimes.shape, crimes.drop_duplicates().shape)
crimes = crimes.drop_duplicates() #It has 23 duplicates.

Let's see how many missing values that we have.

In [ ]:
import missingno as msno
msno.matrix(crimes)

In [ ]:
print(crimes.isnull().sum(), end = '\n\n')
print(crimes[(crimes['lat'].isnull()) | (crimes['long'].isnull())]['location'].unique())

At the below cell, we print count of unique offence_code per offense_code_group and sort them descending. We can see some offense_code_group has more than 1 offense_code under their section. The first 10 are printed. We see that Drug Violation offense_code_group has 26 offense_code underneath it.

In [ ]:
off = crimes.groupby('offense_code_group')['offense_code'].nunique().sort_values(ascending = False)
off.to_frame().reset_index()[:10]

# A-1) How has crime changed over the years?

## Visualization:

### Crime counts per year:
Let's first look at how many crimes are in what years.

In [ ]:
y=crimes.year.value_counts()
print(y)

In [ ]:
sns.set(style="whitegrid", color_codes=True)

year_counts = crimes.groupby('year').count()['incident_number'].to_frame().reset_index()
pal = sns.color_palette("GnBu_d", len(year_counts['year']))
rank = year_counts["incident_number"].argsort().argsort() 
sns.set(rc={'figure.figsize':(5,5)})
ax = sns.barplot(x = 'year' , y="incident_number", data = year_counts, palette=np.array(pal[::-1])[rank])

plt.ylim(40000,110000)
plt.show()

* When we examine the graph, we can see that the crimes increased until 2017 and decreased after peaking in 2017.
* The reasons for the decrease in the number of crimes after 2017 may be the regulations or the increase in penalties. However, we can't make a detailed comment without reviewing crime counts per year and month.


### Crime counts per month

In [ ]:
month_counts = crimes.groupby('month').count()['incident_number'].to_frame().reset_index()
sns.set(rc={'figure.figsize':(8,5)})
pal = sns.color_palette("Blues_d", len(month_counts['month']))
rank = month_counts["incident_number"].argsort().argsort() 
ax = sns.barplot(x = 'month' , y="incident_number", data = month_counts,palette=np.array(pal[::-1])[rank])

* We can generally say that crimes are committed mostly in the summer.

### Crime counts per year and month

In [ ]:
plt.figure(figsize=(15,7))
crimes.groupby(['year','month']).count()['incident_number'].plot.bar()

### Crime counts per day

In [ ]:
day_counts = crimes.groupby('day_of_week').count()['incident_number'].to_frame().reset_index()
sns.set(rc={'figure.figsize':(8,8)})
pal = sns.color_palette("ch:3.1,-.2,dark=.2", len(day_counts['day_of_week']))
rank = day_counts["incident_number"].argsort().argsort() 
ax = sns.barplot(x = 'day_of_week' , y="incident_number", data = day_counts, palette=np.array(pal[::])[rank])

* In terms of day, we can say that the most crime was committed on Fridays and on the weekend.

### Hour incident counts

In [ ]:
hour_nums = crimes.groupby(['hour']).count()['incident_number'].to_frame().reset_index()
sns.set(rc={'figure.figsize':(20,6)})
pal = sns.color_palette("ch:3.2,-.2,dark=.2", len(hour_nums['hour']))
rank = hour_nums["incident_number"].argsort().argsort() 
ax = sns.barplot(x = 'hour' , y="incident_number", data = hour_nums, palette=np.array(pal[::])[rank])

# ax.set_xticklabels(ax.get_xticklabels(),rotation=90); #x eks.deki değerleri yatay yazar.

* We can see that there is an increase in crime during the hours when people continue their daily lives.
* For example, while there is an increase in the rush hours (on the way home from work), a more sluggish situation is observed between 3-6 AM.

### District incident counts

In [ ]:
district_nums =  crimes.groupby('district').count()['incident_number'].sort_values(ascending = False).to_frame().reset_index()
sns.set(rc={'figure.figsize':(7,5)})
pal = sns.color_palette("ch:3.4,-.2,dark=.2", len(district_nums['district']))
rank = district_nums["incident_number"].argsort().argsort() 
ax = sns.barplot(x = 'district' , y="incident_number", data = district_nums, palette=np.array(pal[::])[rank])

# A-2) Answering the Question

Printing & plotting the count of incidents per year, first thing we see is that the counts of the reports has increased almost two times from 2015 to 2016 and almost didn't increase from 2016 to 2017 and finally has dropped around 25 percent from 2017 to 2018.

<br>But this is just the surface and might be misleading. We shall dive deep and get more information before we make a call.

In [ ]:
print(crimes.groupby('year').count()['incident_number'])
sns.set(rc={'figure.figsize':(5,5)})
pal = sns.color_palette("ch:3.5,-.2,dark=.2", len(year_counts['year']))
rank = year_counts["incident_number"].argsort().argsort() 
ax = sns.barplot(x = 'year' , y="incident_number", data = year_counts, palette=np.array(pal[::])[rank])


*  When we groupby our data based on year and get count of months for each year, we see that 2015 has only 7 months and and 2018 has 10. 2016 and 2017 has all 12 months. 
*  This kind of explains why 2015 was almost half of the 2016 and 2018 was around 3/4 of the 2017. The missing months cause a wrong assumption if we don't be cautious. 
* At the next step we find the monthly average incident per year by dividing count of incident to number of months for each year. We see that there actually isn't much  change when we get an average of months. ahere is an increase from 2015 to 2016 and a little increase as well from 2016 to 2017 and finally a decrease from 2017 to 2018.  
*  But this also may be misleading because we don't know if the first month of 2016 which is month 6, has all the days also the last month of 2018 which is month 10, has all the days as well .
*  Therefore we may better get the count of the days and find the daily average of incidents per year

In [ ]:
print('Count of Months Per Year:\n',crimes.groupby('year')['month'].nunique())
av_month = (crimes.groupby('year').count()['incident_number'] / crimes.groupby('year')['month'].nunique()).to_frame().reset_index()
print('\nAverage monthly incident per year:\n',av_month)
av_month.rename(columns = {0:'incident_number'}, inplace = True)

pal = sns.color_palette("ch:3.7,-.3,dark=.3", len(year_counts['year']))
rank = av_month["incident_number"].argsort().argsort() 
sns.set(rc={'figure.figsize':(5,5)})
ax = sns.barplot(x = 'year' , y="incident_number", data = av_month, palette=np.array(pal[::])[rank])

* When we print first and last occurred_on_date we see that the records starts with 15th of the month 6 for year 2015 which means there are missing records for month 6 and we also see that there are only 3 days from month 10 of year 2018. We then find the days for each year and use it to calculate daily count of incidents per year.
* When we plot the average daily incidents we see that they are quite close to each other. The daily average increases a couple of incidents from 2015 to 2016 and increase a couple from 2016 to 2017 as well while there is around 7 incident decrease from 2017 to 2018.

In [ ]:
print(crimes.min()['occurred_on_date'])
print(crimes.max()['occurred_on_date'])

* Let's look at how many days we collect data over the years.**

In [ ]:
crimes['occurred_on_date'] = pd.to_datetime(crimes['occurred_on_date'])
yearly_counts = crimes.groupby('year').count()['incident_number'].to_numpy()
days = []
for year in crimes.year.sort_values().unique():
    days.append((crimes[crimes['year'] == year].max().occurred_on_date - crimes[crimes['year'] == year].min().occurred_on_date).days)

average_daily_incidents = yearly_counts / days  
    
print([str(year)+ ": "+ str(avg)[:4] for year, avg in enumerate(average_daily_incidents, 2015)])

* When we first examined the data in the upper part, we saw that the number of crimes is very different from each other annually. However, when the daily average of incidents per year is examined, we see that the number of crimes committed has not changed much annually.

In [ ]:
d_avg = pd.DataFrame(data = average_daily_incidents, index = av_month.index)
d_avg.rename(columns = {0:'counts'}, inplace = True)

pal = sns.color_palette("ch:3.8,-.1,dark=.2", len(year_counts['year']))
rank = av_month["incident_number"].argsort().argsort() 
sns.set(rc={'figure.figsize':(5,5)})
ax = sns.barplot(x = d_avg.index , y="counts", data = d_avg, palette=np.array(pal[::])[rank])
ax.set_xticklabels(crimes.year.sort_values().unique());

* Below we can see the count of the incidents which shooting was involved. We can see that the shootings increases from 2015 to 2016 and from 2016 to 2017 and then there is a decrease from 2017 to 2018. 
* We of course need to take into consideration that 2015 is almost half year and 2018 is almost 3 months missing. But the increase from 2016 to 2017 is clear as both years are completed

In [ ]:
shooting_years

In [ ]:
shooting = crimes.dropna(subset = ['shooting'])
shooting_years = shooting.groupby('year').count()['incident_number'].to_frame().reset_index()

sns.set(rc={'figure.figsize':(5,5)})
pal = sns.color_palette("ch:4.2,-.1,dark=.2, light=.8", len(shooting_years['year']))

sns.set(rc={'figure.figsize':(5,5)})
ax = sns.barplot(x = shooting_years["year"] , y="incident_number", data = shooting_years, palette=np.array(pal[::])[rank])

* We find the offense_code_group with most incidents over the years and plot the change of it in chart. As we can see, the two complete years have close counts similar to their total count of incidents while the two uncomplete year have less count

In [ ]:
# Now let's look at the top 10 places to see where the crime was committed the most.
""" stmax = df.STREET.value_counts().head(10)
print(stmax)
print(stmax["CENTRE ST"])"""

In [ ]:
# Now let's visualize it.
''' sns.catplot(y='STREET',
           kind='count',
           height=7, 
           aspect=2,
           order=df.STREET.value_counts().head(10).index,
           data=df) '''

In [ ]:
# In the graph below, we see the 30 most common crimes in the city. Top 3 crimes are: Motor Vehicle,  Accident Response, Larceny, Medical Assistance.
''' df2 = pd.DataFrame(columns = ['offenses'])
df2["offenses"]=[each for each in crimes.offense_code_group.unique()]
df2["count"]=[len(crimes[crimes.offense_code_group==each]) for each in df2.offenses]
df2=df2.sort_values(by=['count'],ascending=False)

plt.figure(figsize=(25,15))
sns.barplot(x=df2.offenses.head(30), y=df2.value_counts().head(30)
plt.xticks(rotation= 90)
plt.xlabel('offenses')
plt.ylabel('count')
plt.show() '''